In [ ]:
import requests
import datetime
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

In [ ]:
BASE_URL_CLIMA = "https://api.open-meteo.com/v1/forecast"
BASE_URL_IP = "http://ip-api.com/json/"

In [62]:
def get_clima_historico(lat, lon, date, time):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": ["temperature_2m", "precipitation"],
        "start_date": date,
        "end_date": date,
    }
    responses = openmeteo.weather_api(BASE_URL_CLIMA, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()

    hourly_data = pd.DataFrame()

    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["precipitation"] = hourly_precipitation

    hourly_dataframe = pd.DataFrame(data = hourly_data)
    return hourly_dataframe[hourly_dataframe.index == time]

In [63]:
ip = "189.57.250.90"
url_ip = f"{BASE_URL_IP}{ip}"

response_ip = requests.get(url_ip)
txt_response = response_ip.json()

lat, lon = txt_response["lat"], txt_response["lon"]
datetime_now = datetime.datetime.now()
print(f"Lat: {lat}, Lon: {lon}, DateTime: {datetime_now}")

Lat: -23.6293, Lon: -46.6351, DateTime: 2026-02-11 08:47:19.603520


In [64]:
date_now = datetime_now.date().isoformat()
time_now = datetime_now.time().hour
time_now

8

In [65]:
get_clima_historico(lat, lon, date_now, time_now)

,temperature_2m,precipitation
8,20.058001,0.0
